In [2]:
! pip install langchain_astradb langchain_core langchain_openai datasets transformers

In [3]:
import os
from getpass import getpass


In [4]:
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")
os.environ["ASTRA_DB_API_ENDPOINT"] = input("ASTRA_DB_API_ENDPOINT = ")

os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY = ")

ASTRA_DB_APPLICATION_TOKEN = ··········
ASTRA_DB_API_ENDPOINT = https://c1c4cc44-c030-4253-b7f9-cbc6b6609fd1-us-east-2.apps.astra.datastax.com
OPENAI_API_KEY = ··········


In [5]:
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from datasets import load_dataset

In [6]:
ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.environ.get("ASTRA_DB_API_ENDPOINT")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [25]:
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="orca2000",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

In [8]:
orca_dataset = load_dataset("microsoft/orca-math-word-problems-200k", split='train')

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

In [18]:
docs = []
for entry in orca_dataset:
    # Add a LangChain document with the answer and metadata tags
    content = f"Question: {entry['question']}\nAnswer: {entry['answer']}"
    doc = Document(page_content=content, metadata={"answer": entry["answer"]})
    docs.append(doc)

In [19]:
docs = docs[:2000]

In [26]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 2000 documents.


In [38]:
def retrieve_documents(query):
    results = vstore.similarity_search(query, k=1)
    return results

In [27]:
results = vstore.similarity_search("Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.", k=3)
for res in results:
    print(f"* {res.page_content} + {res.metadata}")

* Question: Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
Answer: If Jungkook is in 5th place, then 4 people crossed the finish line faster than him. + {'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}
* Question: Jungkook, Taehyung, and Minyoung had a running race. Jungkook arrived at the finish line first, followed by Minyoung and Taehyung. What rank does Taehyung in?
Answer: Taehyung came in third place. + {'answer': 'Taehyung came in third place.'}
* Question: Jimin, Jungkook, and Taehyung had a running competition, and as a result, Jimin came in 6th place and Jungkook came in 8th place. Taehyung did better than Jungkook but not when compared with Jimin. What is Taehyung's rank?
Answer: If Taehyung did better than Jungkook, who came in 8th place, but not as well as Jimin, who came in 6th place, then Taehyung must have come in 7th place. + {'answer': 'If Taehyung did better than 

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [23]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model.cuda()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [41]:
def generate_prompt(content, query):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    SYS_PROMPT = B_SYS + "Use the following context to help answer the user question. Only provide the answer, nothing more." + E_SYS

    instruction = f"Context: {content} User Question: {query}"

    prompt_template = B_INST + SYS_PROMPT + instruction + E_INST
    return prompt_template



def chat_with_llama(content, query):

    def cut_off_text(text, prompt):
        cutoff_phrase = prompt
        index = text.find(cutoff_phrase)
        if index != -1:
          return text[:index]
        else:
          return text

    def remove_substring(string, substring):
        return string.replace(substring, "")

    prompt = generate_prompt(content, query)

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda')
    output = model.generate(input_ids)
    response = tokenizer.batch_decode(output, skip_special_tokens=True)
    response = cut_off_text(response[0], '</s>')
    response = remove_substring(response, prompt)
    return response


In [42]:
def interactive_chat():
    print("LLaMA 2 Chatbot: I'm here to help with math problems. Ask me anything!")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("LLaMA 2 Chatbot: Goodbye!")
            break
        documents = retrieve_documents(user_input)
        context = " ".join([doc.page_content for doc in documents])
        response = chat_with_llama(context, user_input)
        print("LLaMA 2 Chatbot:", response)

In [43]:
interactive_chat()

LLaMA 2 Chatbot: I'm here to help with math problems. Ask me anything!
You: Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
LLaMA 2 Chatbot:   Sure! Here is the answer to the user question:

4 people
You: The length of one span of Jinseo is about 10 centimeters (cm). When Jinseo measured the length of the shorter side of the bookshelf, it was about two spans. How many centimeters (cm) is the short side of the bookshelf?
LLaMA 2 Chatbot:   Sure! Here is the answer to the user question:

2 spans * 10 cm/span = 20 cm
You: What is the difference between the largest number and the smallest number that can be made by selecting four from the number magnets 2, 0, 3, 5, and 8 and arranging them in a row to form a four-digit number?
LLaMA 2 Chatbot:   The difference between the largest and smallest four-digit numbers that can be made by selecting four from the number magnets 2, 0, 3, 5, and 8 and arranging them in a row is 6497.
You: exit
LL